In [1]:
import pandas as pd
from datetime import date
import datetime
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from gensim.models import Word2Vec
import os
import multiprocessing as mp
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# Item2vec Parameters

In [2]:
import json

def save_json(data,name):
    with open(name+".json","w") as f:
        json.dump(data,f, ensure_ascii = False)
        
def load_json(path):
    f = open(path,"r")
    data = json.loads(f.read())
    f.close()
    return data

In [3]:
products = pd.read_csv("/kaggle/input/shwapno-small/products_small.csv")
products.columns = products.iloc[0].to_list()
products = products.drop([0])
products = products.reset_index(drop=True)

product_code2name = dict(zip(list(products["ProductCode"]),list(products["ProductName"])))
product2price = dict(zip(list(products["ProductCode"]),list(products["UnitPrice"])))

invoices = pd.read_csv("/kaggle/input/shwapno-small/invoices_small_jan23.csv")
customers = pd.read_csv("/kaggle/input/shwapno-small/customer_small.csv")


/tmp/ipykernel_32/941033493.py:1: DtypeWarning: Columns (0,2,4,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  products = pd.read_csv("/kaggle/input/shwapno-small/products_small.csv")


In [4]:
invoices_2 = invoices.drop(columns = ["ProductNSI","NSI","SalesQTY","Cash_1_Card_0","InvoicePeriod","PrepareDate","DepotCode"])
invoices_2

,CustomerCode,ProductCode,InvoiceNo,InvoiceDate
0,1301722013,2300011,D0922302060241,2023-02-06 00:00:00.000
1,1301722013,2300011,D0922301160317,2023-01-16 00:00:00.000
2,1301722013,2300011,D0922302280287,2023-02-28 00:00:00.000
3,1301722013,2300139,D0922303280358,2023-03-28 00:00:00.000
4,1301722013,2300438,D0922303280358,2023-03-28 00:00:00.000
...,...,...,...,...
3860886,99084536,3500122,D0542304150067,2023-04-15 00:00:00.000
3860887,99084536,3391176,D0542303150199,2023-03-15 00:00:00.000
3860888,99084536,3103285,D0542303090198,2023-03-09 00:00:00.000
3860889,99084536,3103286,D0542303090198,2023-03-09 00:00:00.000


In [5]:
invoices_3 = invoices_2.groupby("InvoiceNo").agg(list)
invoices_3

,CustomerCode,ProductCode,InvoiceDate
InvoiceNo,,,
B0042304300006,[1786444466],[2815047],[2023-04-30 00:00:00.000]
B0042305010016,[1772285528],[2801885],[2023-05-01 00:00:00.000]
B0042305030059,[1717457631],[2304304],[2023-05-03 00:00:00.000]
B0042305030087,[1786444466],[2815516],[2023-05-03 00:00:00.000]
B0042305040006,"[1783561341, 1783561341, 1783561341]","[2400643, 2603818, 2603741]","[2023-05-04 00:00:00.000, 2023-05-04 00:00:00...."
...,...,...,...
S0092306290004,"[1712182064, 1712182064]","[3012857, 2400217]","[2023-06-29 00:00:00.000, 2023-06-29 00:00:00...."
S0092306290006,"[1712182064, 1712182064]","[2815013, 2813317]","[2023-06-29 00:00:00.000, 2023-06-29 00:00:00...."
S0092306300001,"[1712265658, 1712265658, 1712265658, 171226565...","[2800038, 2801004, 2700060, 2400136, 2500099, ...","[2023-06-30 00:00:00.000, 2023-06-30 00:00:00...."


In [6]:
invoices_4 = invoices_3.copy().reset_index()
invoices_4.head()

,InvoiceNo,CustomerCode,ProductCode,InvoiceDate
0,B0042304300006,[1786444466],[2815047],[2023-04-30 00:00:00.000]
1,B0042305010016,[1772285528],[2801885],[2023-05-01 00:00:00.000]
2,B0042305030059,[1717457631],[2304304],[2023-05-03 00:00:00.000]
3,B0042305030087,[1786444466],[2815516],[2023-05-03 00:00:00.000]
4,B0042305040006,"[1783561341, 1783561341, 1783561341]","[2400643, 2603818, 2603741]","[2023-05-04 00:00:00.000, 2023-05-04 00:00:00...."


In [7]:
invoices_4["CustomerCode"] = invoices_4["CustomerCode"].apply(lambda x: ["C-"+str(x[0])])
invoices_4["InvoiceDate"] = invoices_4["InvoiceDate"].apply(lambda x: [x[0].split(" ")[0]])
invoices_4["ProductCode"] = invoices_4["ProductCode"].apply(lambda x: ["P-"+str(i) for i in x])
invoices_4.head()

,InvoiceNo,CustomerCode,ProductCode,InvoiceDate
0,B0042304300006,[C-1786444466],[P-2815047],[2023-04-30]
1,B0042305010016,[C-1772285528],[P-2801885],[2023-05-01]
2,B0042305030059,[C-1717457631],[P-2304304],[2023-05-03]
3,B0042305030087,[C-1786444466],[P-2815516],[2023-05-03]
4,B0042305040006,[C-1783561341],"[P-2400643, P-2603818, P-2603741]",[2023-05-04]


In [8]:
invoices_4["basket"] = invoices_4["CustomerCode"]+invoices_4["InvoiceDate"]+invoices_4["ProductCode"]
invoices_4.head()

,InvoiceNo,CustomerCode,ProductCode,InvoiceDate,basket
0,B0042304300006,[C-1786444466],[P-2815047],[2023-04-30],"[C-1786444466, 2023-04-30, P-2815047]"
1,B0042305010016,[C-1772285528],[P-2801885],[2023-05-01],"[C-1772285528, 2023-05-01, P-2801885]"
2,B0042305030059,[C-1717457631],[P-2304304],[2023-05-03],"[C-1717457631, 2023-05-03, P-2304304]"
3,B0042305030087,[C-1786444466],[P-2815516],[2023-05-03],"[C-1786444466, 2023-05-03, P-2815516]"
4,B0042305040006,[C-1783561341],"[P-2400643, P-2603818, P-2603741]",[2023-05-04],"[C-1783561341, 2023-05-04, P-2400643, P-260381..."


In [9]:
def sort_by_priority_list(values, priority):
    priority_dict = dict(
        zip(
            priority,
            range(len(priority)),
        ),
    )
    priority_getter = priority_dict.get # dict.get(key)
    return sorted(values, key=priority_getter)


In [10]:
invoices_33 = invoices_2.drop(columns = ["InvoiceDate","InvoiceNo"]).groupby("CustomerCode").agg(list).reset_index()
invoices_33["ProductCode"] = invoices_33["ProductCode"].apply(lambda x: ["P-"+str(i) for i in x])
invoices_33["Priority"] = invoices_33["ProductCode"].apply(lambda x: sorted(x, key = x.count,reverse = True))
invoices_33["CustomerCode"] = invoices_33["CustomerCode"].apply(lambda x: "C-"+str(x))
invoices_33.head()

,CustomerCode,ProductCode,Priority
0,C-2213067,"[P-2600078, P-2600276, P-2603742, P-2603189, P...","[P-2400212, P-2400212, P-2400212, P-2400212, P..."
1,C-3074407,"[P-2901007, P-3101680, P-2817453, P-2816443, P...","[P-3102046, P-3102046, P-3102046, P-3102046, P..."
2,C-3074509,"[P-3810689, P-3500081, P-3102825, P-6000140, P...","[P-3100617, P-3100617, P-3100617, P-3100617, P..."
3,C-3077523,"[P-3101610, P-2808944, P-2808794, P-2809952, P...","[P-3101784, P-3101784, P-3101784, P-3101784, P..."
4,C-3260206,"[P-2808120, P-2901593, P-3009733, P-2815517, P...","[P-2901484, P-2901484, P-2901484, P-6000135, P..."


In [11]:
priority_dict = dict(zip(list(invoices_33["CustomerCode"]),list(invoices_33["Priority"])))

In [12]:
invoices_4["basket_sorted"] = invoices_4["basket"].apply(lambda x: x[0:2]+sort_by_priority_list(x[2:], priority_dict[x[0]]))
invoices_4

,InvoiceNo,CustomerCode,ProductCode,InvoiceDate,basket,basket_sorted
0,B0042304300006,[C-1786444466],[P-2815047],[2023-04-30],"[C-1786444466, 2023-04-30, P-2815047]","[C-1786444466, 2023-04-30, P-2815047]"
1,B0042305010016,[C-1772285528],[P-2801885],[2023-05-01],"[C-1772285528, 2023-05-01, P-2801885]","[C-1772285528, 2023-05-01, P-2801885]"
2,B0042305030059,[C-1717457631],[P-2304304],[2023-05-03],"[C-1717457631, 2023-05-03, P-2304304]","[C-1717457631, 2023-05-03, P-2304304]"
3,B0042305030087,[C-1786444466],[P-2815516],[2023-05-03],"[C-1786444466, 2023-05-03, P-2815516]","[C-1786444466, 2023-05-03, P-2815516]"
4,B0042305040006,[C-1783561341],"[P-2400643, P-2603818, P-2603741]",[2023-05-04],"[C-1783561341, 2023-05-04, P-2400643, P-260381...","[C-1783561341, 2023-05-04, P-2400643, P-260381..."
...,...,...,...,...,...,...
864164,S0092306290004,[C-1712182064],"[P-3012857, P-2400217]",[2023-06-29],"[C-1712182064, 2023-06-29, P-3012857, P-2400217]","[C-1712182064, 2023-06-29, P-2400217, P-3012857]"
864165,S0092306290006,[C-1712182064],"[P-2815013, P-2813317]",[2023-06-29],"[C-1712182064, 2023-06-29, P-2815013, P-2813317]","[C-1712182064, 2023-06-29, P-2815013, P-2813317]"
864166,S0092306300001,[C-1712265658],"[P-2800038, P-2801004, P-2700060, P-2400136, P...",[2023-06-30],"[C-1712265658, 2023-06-30, P-2800038, P-280100...","[C-1712265658, 2023-06-30, P-2401040, P-301616..."
864167,S0092306300003,[C-1316157872],"[P-2816757, P-2802437, P-2801649, P-2702600, P...",[2023-06-30],"[C-1316157872, 2023-06-30, P-2816757, P-280243...","[C-1316157872, 2023-06-30, P-2816757, P-280243..."


In [13]:
invoices_4.to_csv("new_sorted_basket.csv", index=False)
